In [1]:
import boto3
import uuid
import logging 

session = boto3.session.Session()
sts_client = boto3.client('sts')
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
agent_id = "PGJCAIWRJV"
agent_alias_id = "9RPZ0M5SKC"

user_query = """do you have any 2 bed rooms from 14th June to 21st June 2025?"""

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText=user_query,
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)
event_stream = agentResponse['completion']

try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-04-22 11:42:25,833] p90327 {1918014581.py:24} INFO - Final answer ->
I'm sorry, but we don't have any 2-bed rooms available from June 14th to June 21st, 2025. Would you like to check availability for different dates or a different room type?
